In [1]:
import paddle
import paddle_quantum
import numpy
import math
import time
import os
from paddle import matmul
from paddle_quantum.linalg import dagger
from paddle_quantum.ansatz import Circuit
from paddle_quantum.qinfo import pauli_str_to_matrix

# Hardware efficient ansatz
def U_hardware(theta, num_qubits, depth):

    # Initialize quantum circuit
    circuit = Circuit(num_qubits)
    
    for j in range(depth):
        circuit.ry(qubits_idx='full', param=theta[j, :, 0])
        circuit.rz(qubits_idx='full', param=theta[j, :, 1])

        circuit.h(qubits_idx=1)
        circuit.cnot(qubits_idx=[0,1])
        circuit.h(qubits_idx=1)

        circuit.h(qubits_idx=2)
        circuit.cnot(qubits_idx=[1,2])
        circuit.h(qubits_idx=2)

        circuit.h(qubits_idx=3)
        circuit.cnot(qubits_idx=[2,3])
        circuit.h(qubits_idx=3)

    return circuit

def symmetry_preserving_type1(circuit, theta, j):
    circuit.cnot(qubits_idx=[[1, 0], [3, 2]])

    circuit.ry(qubits_idx=[1, 3], param=-(theta[j, 0:2, 0] + math.pi/2))
    circuit.rz(qubits_idx=[1, 3], param=-(theta[j, 0:2, 1] + math.pi))

    circuit.cnot(qubits_idx=[[0,1], [2,3]])

    circuit.rz(qubits_idx=[1, 3], param=theta[j, 0:2, 1] + math.pi)
    circuit.ry(qubits_idx=[1, 3], param=theta[j, 0:2, 0] + math.pi/2)

    circuit.cnot(qubits_idx=[[1,0], [3,2]])


    circuit.cnot(qubits_idx=[2, 1])

    circuit.ry(qubits_idx=2, param=-(theta[j, 2, 2] + math.pi/2))
    circuit.rz(qubits_idx=2, param=-(theta[j, 2, 3] + math.pi))

    circuit.cnot(qubits_idx=[1,2])

    circuit.rz(qubits_idx=2, param=theta[j, 2, 3] + math.pi)
    circuit.ry(qubits_idx=2, param=theta[j, 2, 2] + math.pi/2)

    circuit.cnot(qubits_idx=[2, 1])


    circuit.cnot(qubits_idx=[[1, 0], [3, 2]])

    circuit.ry(qubits_idx=[1, 3], param=-(theta[j, 0:2, 4] + math.pi/2))
    circuit.rz(qubits_idx=[1, 3], param=-(theta[j, 0:2, 5] + math.pi))

    circuit.cnot(qubits_idx=[[0,1], [2,3]])

    circuit.rz(qubits_idx=[1, 3], param=theta[j, 0:2, 5] + math.pi)
    circuit.ry(qubits_idx=[1, 3], param=theta[j, 0:2, 4] + math.pi/2)

    circuit.cnot(qubits_idx=[[1,0], [3,2]])


    circuit.cnot(qubits_idx=[2, 1])

    circuit.ry(qubits_idx=2, param=-(theta[j, 2, 6] + math.pi/2))
    circuit.rz(qubits_idx=2, param=-(theta[j, 2, 7] + math.pi))

    circuit.cnot(qubits_idx=[1,2])

    circuit.rz(qubits_idx=2, param=theta[j, 2, 7] + math.pi)
    circuit.ry(qubits_idx=2, param=theta[j, 2, 6] + math.pi/2)

    circuit.cnot(qubits_idx=[2, 1])

    return circuit

def symmetry_preserving_type2(circuit, theta, j):
    circuit.rz(qubits_idx=[1, 3], param=math.pi/2)
    
    circuit.cnot(qubits_idx=[[1, 0], [3, 2]])

    circuit.rz(qubits_idx=[0, 2], param=2 * theta[j, 0:2, 0] - math.pi/2)
    circuit.ry(qubits_idx=[1, 3], param=math.pi/2 - 2 * theta[j, 0:2, 0])

    circuit.cnot(qubits_idx=[[0,1], [2,3]])

    circuit.ry(qubits_idx=[1, 3], param=2 * theta[j, 0:2, 0] - math.pi/2)

    circuit.cnot(qubits_idx=[[1,0], [3,2]])

    circuit.rz(qubits_idx=[0, 2], param=-math.pi/2)


    circuit.rz(qubits_idx=2, param=math.pi/2)
    
    circuit.cnot(qubits_idx=[2, 1])

    circuit.rz(qubits_idx=1, param=2 * theta[j, 0, 1] - math.pi/2)
    circuit.ry(qubits_idx=2, param=math.pi/2 - 2 * theta[j, 0, 1])

    circuit.cnot(qubits_idx=[1,2])

    circuit.ry(qubits_idx=2, param=2 * theta[j, 0, 1] - math.pi/2)

    circuit.cnot(qubits_idx=[2, 1])

    circuit.rz(qubits_idx=1, param=-math.pi/2)

    return circuit

# Symmetry ansatz quantum circuit type 1
def U_NumOp_type1(theta, num_qubits, depth):

    # Initialize quantum circuit
    circuit = Circuit(num_qubits)
    
    for j in range(depth):
        symmetry_preserving_type1(circuit, theta, j)

    return circuit

# Symmetry ansatz quantum circuit type 2
def U_NumOp_type2(theta, num_qubits, depth):

    # Initialize quantum circuit
    circuit = Circuit(num_qubits)
    
    for j in range(depth):
        symmetry_preserving_type2(circuit, theta, j)

    return circuit

class VQE_symmetry_class(paddle.nn.Layer):
    def __init__(self, depth, width, length_layer):
        super(VQE_symmetry_class, self).__init__()

        # Random parameters
        Uni_ini = paddle.nn.initializer.Uniform(low=0.0, 
                                                high=2 * math.pi)
        
        self.theta = self.create_parameter(shape=[depth, width, length_layer],
                                           default_initializer=Uni_ini,
                                           dtype='float64')
        
    def forward(self, process_selection, H, NumOp, Sz):
        if process_selection in [Hea_g, Hea_ae, Hea_ae_Sz]:
            cir = U_hardware(self.theta, N, D)
        elif process_selection == Spat1_g:
            cir = U_NumOp_type1(self.theta, N, D)
        elif process_selection in [Spat2_g, Spat2_en2, Spat2_en1, 
                                   Spat2_en0, Spat2_en3, Spat2_en4]:
            cir = U_NumOp_type2(self.theta, N, D)
        
        # Matrix transformation
        U = cir.unitary_matrix()
        H_U = matmul(H, U)
        NumOp_U = matmul(NumOp, U)
        Sz_U = matmul(Sz, U)
        
        Udagger_H_U = matmul(dagger(U), H_U)
        Udagger_NumOp_U = matmul(dagger(U), NumOp_U)
        Udagger_Sz_U = matmul(dagger(U), Sz_U)
        
        H_trans = paddle.real(Udagger_H_U)
        NumOp_trans = paddle.real(Udagger_NumOp_U)
        Sz_trans = paddle.real(Udagger_Sz_U)

        # Define coefficient sign
        def pm(x):
            if x == 0:
                return 1
            elif x == 1:
                return -1
        
        def pmpm(x):
            if x == 0 or x == 2:
                return 1
            elif x == 1 or x == 3:
                return -1

        def ppmm(x):
            if x == 0 or x == 1:
                return 1
            elif x == 2 or x == 3:
                return -1
        
        def pmmp(x):
            if x == 0 or x == 3:
                return 1
            elif x == 1 or x == 2:
                return -1
        
        # Compute the expectation value of the operator after the initial state undergoes quantum gate operations
        def compute_expectation(basis_states, trans_matrix, coeff):
            exp_psi = 0
            for i in range(len(basis_states)):
                for j in range(len(basis_states)):
                    exp_psi += coeff(i, j) * trans_matrix[basis_states[i]][basis_states[j]]
            return exp_psi

        # The corresponding coefficients of different initial states during the compute process
        def coeff0(i, j): return 1/4
        def coeff1(i, j): return 1/2
        def coeff2(i, j): return 1/2
        def coeff3(i, j): return 1
        def coeff4(i, j): return 1
        def coeff5(i, j): return 1
        def coeff6(i, j): return 1/4 * pmpm(i) * pmpm(j)
        def coeff7(i, j): return 1/2
        def coeff8(i, j): return 1/2 * pm(i) * pm(j)
        def coeff9(i, j): return 1/2 * pm(i) * pm(j)
        def coeff10(i, j): return 1/2
        def coeff11(i, j): return 1/4 * ppmm(i) * ppmm(j)
        def coeff12(i, j): return 1/2 * pm(i) * pm(j)
        def coeff13(i, j): return 1/2 * pm(i) * pm(j)
        def coeff14(i, j): return 1/4 * pmmp(i) * pmmp(j)
        def coeff15(i, j): return 1

        # The expectation value of the Hamiltonian under different initial states
        H0_psi = compute_expectation(g_values, H_trans, coeff0)
        H1_psi = compute_expectation(e1_values, H_trans, coeff1)
        H2_psi = compute_expectation(e2_values, H_trans, coeff2)
        H3_psi = compute_expectation(e3_values, H_trans, coeff3)
        H4_psi = compute_expectation(e4_values, H_trans, coeff4)
        H5_psi = compute_expectation(e5_values, H_trans, coeff5)
        H6_psi = compute_expectation(e6_values, H_trans, coeff6)
        H7_psi = compute_expectation(e7_values, H_trans, coeff7)
        H8_psi = compute_expectation(e8_values, H_trans, coeff8)
        H9_psi = compute_expectation(e9_values, H_trans, coeff9)
        H10_psi = compute_expectation(e10_values, H_trans, coeff10)
        H11_psi = compute_expectation(e11_values, H_trans, coeff11)
        H12_psi = compute_expectation(e12_values, H_trans, coeff12)
        H13_psi = compute_expectation(e13_values, H_trans, coeff13)
        H14_psi = compute_expectation(e14_values, H_trans, coeff14)
        H15_psi = compute_expectation(e15_values, H_trans, coeff15)

        # The expectation value of the particle number operator under different initial states
        NumOp0_psi = compute_expectation(g_values, NumOp_trans, coeff0)
        NumOp1_psi = compute_expectation(e1_values, NumOp_trans, coeff1)
        NumOp2_psi = compute_expectation(e2_values, NumOp_trans, coeff2)
        NumOp3_psi = compute_expectation(e3_values, NumOp_trans, coeff3)
        NumOp4_psi = compute_expectation(e4_values, NumOp_trans, coeff4)
        NumOp5_psi = compute_expectation(e5_values, NumOp_trans, coeff5)
        NumOp6_psi = compute_expectation(e6_values, NumOp_trans, coeff6)
        NumOp7_psi = compute_expectation(e7_values, NumOp_trans, coeff7)
        NumOp8_psi = compute_expectation(e8_values, NumOp_trans, coeff8)
        NumOp9_psi = compute_expectation(e9_values, NumOp_trans, coeff9)
        NumOp10_psi = compute_expectation(e10_values, NumOp_trans, coeff10)
        NumOp11_psi = compute_expectation(e11_values, NumOp_trans, coeff11)
        NumOp12_psi = compute_expectation(e12_values, NumOp_trans, coeff12)
        NumOp13_psi = compute_expectation(e13_values, NumOp_trans, coeff13)
        NumOp14_psi = compute_expectation(e14_values, NumOp_trans, coeff14)
        NumOp15_psi = compute_expectation(e15_values, NumOp_trans, coeff15)

        # The expectation value of the total spin z component operator under different initial states
        Sz0_psi = compute_expectation(g_values, Sz_trans, coeff0)
        Sz1_psi = compute_expectation(e1_values, Sz_trans, coeff1)
        Sz2_psi = compute_expectation(e2_values, Sz_trans, coeff2)
        Sz3_psi = compute_expectation(e3_values, Sz_trans, coeff3)
        Sz4_psi = compute_expectation(e4_values, Sz_trans, coeff4)
        Sz5_psi = compute_expectation(e5_values, Sz_trans, coeff5)
        Sz6_psi = compute_expectation(e6_values, Sz_trans, coeff6)
        Sz7_psi = compute_expectation(e7_values, Sz_trans, coeff7)
        Sz8_psi = compute_expectation(e8_values, Sz_trans, coeff8)
        Sz9_psi = compute_expectation(e9_values, Sz_trans, coeff9)
        Sz10_psi = compute_expectation(e10_values, Sz_trans, coeff10)
        Sz11_psi = compute_expectation(e11_values, Sz_trans, coeff11)
        Sz12_psi = compute_expectation(e12_values, Sz_trans, coeff12)
        Sz13_psi = compute_expectation(e13_values, Sz_trans, coeff13)
        Sz14_psi = compute_expectation(e14_values, Sz_trans, coeff14)
        Sz15_psi = compute_expectation(e15_values, Sz_trans, coeff15)

        H_psi = [H0_psi, H1_psi, H2_psi, H3_psi, 
                 H4_psi, H5_psi, H6_psi, H7_psi, 
                 H8_psi, H9_psi, H10_psi, H11_psi, 
                 H12_psi, H13_psi, H14_psi, H15_psi]
        
        NumOp_psi = [NumOp0_psi, NumOp1_psi, NumOp2_psi, NumOp3_psi, 
                     NumOp4_psi, NumOp5_psi, NumOp6_psi, NumOp7_psi, 
                     NumOp8_psi, NumOp9_psi, NumOp10_psi, NumOp11_psi, 
                     NumOp12_psi, NumOp13_psi, NumOp14_psi, NumOp15_psi]
        
        Sz_psi = [Sz0_psi, Sz1_psi, Sz2_psi, Sz3_psi, 
                  Sz4_psi, Sz5_psi, Sz6_psi, Sz7_psi, 
                  Sz8_psi, Sz9_psi, Sz10_psi, Sz11_psi, 
                  Sz12_psi, Sz13_psi, Sz14_psi, Sz15_psi]
        
        # Loss function
        loss_g = H_psi[0]
        
        loss_hard_state = 0
        for i in range(len(H_psi)):
            omega = len(H_psi) - i
            loss_hard_state += omega * H_psi[i]

        loss_hard_state_Sz = (16 * H_psi[0] + 15 * (H_psi[1] + beta * (Sz_psi[1] + 0.5)**2) 
                              + 14 * (H_psi[2] + beta * (Sz_psi[2] - 0.5)**2) 
                              + 13 * (H_psi[3] + beta * (Sz_psi[3] + 1)**2) 
                              + 12 * H_psi[4] 
                              + 11 * (H_psi[5] + beta * (Sz_psi[5] - 1)**2) 
                              + 10 * H_psi[6] + 9 * (H_psi[7] + beta * (Sz_psi[7] + 0.5)**2) 
                              + 8 * (H_psi[8] + beta * (Sz_psi[8] - 0.5)**2) 
                              + 7 * (H_psi[9] + beta * (Sz_psi[9] + 0.5)**2) 
                              + 6 * (H_psi[10] + beta * (Sz_psi[10] - 0.5)**2)
                              + 5 * H_psi[11] + 4 * (H_psi[12] + beta * (Sz_psi[12] + 0.5)**2) 
                              + 3 * (H_psi[13] + beta * (Sz_psi[13] - 0.5)**2) 
                              + 2 * H_psi[14] + 1 * H_psi[15])

        loss_n2 = (6 * H_psi[0] 
                   + 5 * (H_psi[3] + beta * (Sz_psi[3] + 1)**2) 
                   + 4 * (H_psi[5] + beta * (Sz_psi[5] - 1)**2) 
                   + 3 * H_psi[6] + 2 * H_psi[11] + 1 * H_psi[14])

        loss_n1 = (4 * (H_psi[1] + beta * (Sz_psi[1] + 0.5)**2) 
                   + 3 * (H_psi[2] + beta * (Sz_psi[2] - 0.5)**2)
                   + 2 * (H_psi[8] + beta * (Sz_psi[8] - 0.5)**2)
                   + 1 * (H_psi[9] + beta * (Sz_psi[9] + 0.5)**2))

        loss_n0 = H_psi[4]

        loss_n3 = (4 * (H_psi[7] + beta * (Sz_psi[7] + 0.5)**2)
                   + 3 * (H_psi[10] + beta * (Sz_psi[10] - 0.5)**2)
                   + 2 * (H_psi[12] + beta * (Sz_psi[12] + 0.5)**2)
                   + 1 * (H_psi[13] + beta * (Sz_psi[13] - 0.5)**2))

        loss_n4 = H_psi[15]

        if process_selection in [Hea_g, Spat1_g, Spat2_g]:
            loss = loss_g
        else:
            loss_dict = {
                Hea_ae: loss_hard_state, 
                Hea_ae_Sz: loss_hard_state_Sz, 
                Spat2_en2: loss_n2, 
                Spat2_en1: loss_n1, 
                Spat2_en0: loss_n0, 
                Spat2_en3: loss_n3, 
                Spat2_en4: loss_n4
            }
            loss = loss_dict[process_selection]
        
        return loss, H_psi, NumOp_psi, Sz_psi, cir

Hea_g = 'Hardware efficient ansatz, ground state'
Hea_ae = 'Hardware efficient ansatz, all eigenstates'
Hea_ae_Sz = 'Hardware efficient ansatz, all eigenstates, Sz penalty term'
Spat1_g = 'Symmetry preserving ansatz type 1, ground state'
Spat2_g = 'Symmetry preserving ansatz type 2, ground state'
Spat2_en2 = 'Symmetry preserving ansatz type 2, Eigenstate, n=2'
Spat2_en1 = 'Symmetry preserving ansatz type 2, Eigenstate, n=1'
Spat2_en0 = 'Symmetry preserving ansatz type 2, Eigenstate, n=0'
Spat2_en3 = 'Symmetry preserving ansatz type 2, Eigenstate, n=3'
Spat2_en4 = 'Symmetry preserving ansatz type 2, Eigenstate, n=4'

states_corresponding = {
    Hea_g: [0], 
    Hea_ae: [i for i in range(16)], 
    Hea_ae_Sz: [i for i in range(16)], 
    Spat1_g: [0], 
    Spat2_g: [0], 
    Spat2_en2: [0, 3, 5, 6, 11, 14], 
    Spat2_en1: [1, 2, 8, 9], 
    Spat2_en0: [4], 
    Spat2_en3: [7, 10, 12, 13], 
    Spat2_en4: [15]
}

file_corresponding = {
    'Hardware efficient ansatz, ground state': 'Hea_g', 
    'Hardware efficient ansatz, all eigenstates': 'Hea_ae', 
    'Hardware efficient ansatz, all eigenstates, Sz penalty term': 'Hea_ae_Sz', 
    'Symmetry preserving ansatz type 1, ground state': 'Spat1_g', 
    'Symmetry preserving ansatz type 2, ground state': 'Spat2_g', 
    'Symmetry preserving ansatz type 2, Eigenstate, n=2': 'Spat2_en2', 
    'Symmetry preserving ansatz type 2, Eigenstate, n=1': 'Spat2_en1', 
    'Symmetry preserving ansatz type 2, Eigenstate, n=0': 'Spat2_en0', 
    'Symmetry preserving ansatz type 2, Eigenstate, n=3': 'Spat2_en3', 
    'Symmetry preserving ansatz type 2, Eigenstate, n=4': 'Spat2_en4'
}

# Select the process to compute
proc_sel = input('Select the process you want to compute. You can choose from (copy one of the lines):\n'
                 '\n'
                 f"{Hea_g}\n"
                 f"{Hea_ae}\n"
                 f"{Hea_ae_Sz}\n"
                 f"{Spat1_g}\n"
                 f"{Spat2_g}\n"
                 f"{Spat2_en2}\n"
                 f"{Spat2_en1}\n"
                 f"{Spat2_en0}\n"
                 f"{Spat2_en3}\n"
                 f"{Spat2_en4}\n"
                 '\n'
                 'Note: If any other parameters need to be modified, please adjust them in the code.')

if proc_sel not in states_corresponding:
    print('\n\nInvalid selection. Please choose a valid process.\n\n')
else:
    print(f"You selected: {proc_sel}\n")
    print(f"Program is running... \
    The generated data is saved in the Data_1D_{file_corresponding[proc_sel]} folder on the desktop.\n")

# parameters
t = 1
U = 2
N = 4               # number of qubits
iterations = 300         # number of iterations
Learning_Rate = 0.1      # learning rate
n_file = 200             # number of repeated processes
iter_step_size = 1       # interval of iteration value output
prec = 4            # precision of the observables output
prec_fid = 4        # precision of the fidelity output
beta = 1

if proc_sel in [Hea_g, Hea_ae, Hea_ae_Sz]:
    if proc_sel == Hea_g:
        D = 12      # number of layers in the quantum circuit
    elif proc_sel in [Hea_ae, Hea_ae_Sz]:
        D = 15
        beta = 0.5       # loss function penalty factor
    W = N      # the number of parameters per column in each layer of the quantum circuit
    L = 2      # the number of parameters per row in each layer of the quantum circuit
elif proc_sel == Spat1_g:
    D = 2
    W = int(N/2) + 1
    L = 16
elif proc_sel in [Spat2_g, Spat2_en2, Spat2_en1, 
                  Spat2_en0, Spat2_en3, Spat2_en4]:
    if proc_sel == Spat2_g:
        D = 4
    elif proc_sel in [Spat2_en2, Spat2_en1, 
                    Spat2_en0, Spat2_en3, Spat2_en4]:
        D = 5
        beta = 1
    W = int(N/2)
    L = 2

# Initial states
g_1 = '0101'
g_2 = '0110'
g_3 = '1001'
g_4 = '1010'
g_values = [int(g_1, 2), int(g_2, 2), int(g_3, 2), int(g_4, 2)]
        
e1_1 = '0001'
e1_2 = '0010'
e1_values = [int(e1_1, 2), int(e1_2, 2)]

e2_1 = '0100'
e2_2 = '1000'
e2_values = [int(e2_1, 2), int(e2_2, 2)]

e3_1 = '0011'
e3_values = [int(e3_1, 2)]

e4_1 = '0000'
e4_values = [int(e4_1, 2)]

e5_1 = '1100'
e5_values = [int(e5_1, 2)]

e6_1 = '0101'
e6_2 = '0110'
e6_3 = '1001'
e6_4 = '1010'
e6_values = [int(e6_1, 2), int(e6_2, 2), int(e6_3, 2), int(e6_4, 2)]

e7_1 = '0111'
e7_2 = '1011'
e7_values = [int(e7_1, 2), int(e7_2, 2)]

e8_1 = '0100'
e8_2 = '1000'
e8_values = [int(e8_1, 2), int(e8_2, 2)]

e9_1 = '0001'
e9_2 = '0010'
e9_values = [int(e9_1, 2), int(e9_2, 2)]

e10_1 = '1101'
e10_2 = '1110'
e10_values = [int(e10_1, 2), int(e10_2, 2)]

e11_1 = '0101'
e11_2 = '0110'
e11_3 = '1001'
e11_4 = '1010'
e11_values = [int(e11_1, 2), int(e11_2, 2), int(e11_3, 2), int(e11_4, 2)]

e12_1 = '0111'
e12_2 = '1011'
e12_values = [int(e12_1, 2), int(e12_2, 2)]

e13_1 = '1101'
e13_2 = '1110'
e13_values = [int(e13_1, 2), int(e13_2, 2)]

e14_1 = '0101'
e14_2 = '0110'
e14_3 = '1001'
e14_4 = '1010'
e14_values = [int(e14_1, 2), int(e14_2, 2), int(e14_3, 2), int(e14_4, 2)]

e15_1 = '1111'
e15_values = [int(e15_1, 2)]

# Define the computational basis
basis = numpy.eye(2**N)

# Used when computing fidelity
initial_state_g = 1/2 * (basis[int(g_1, 2)] + basis[int(g_2, 2)] 
                         + basis[int(g_3, 2)] + basis[int(g_4, 2)])

initial_state_e1 = 1/math.sqrt(2) * (basis[int(e1_1, 2)] + basis[int(e1_2, 2)])

initial_state_e2 = 1/math.sqrt(2) * (basis[int(e2_1, 2)] + basis[int(e2_2, 2)])

initial_state_e3 = basis[int(e3_1, 2)]

initial_state_e4 = basis[int(e4_1, 2)]

initial_state_e5 = basis[int(e5_1, 2)]

initial_state_e6 = 1/2 * (basis[int(e6_1, 2)] - basis[int(e6_2, 2)] 
                          + basis[int(e6_3, 2)] - basis[int(e6_4, 2)])

initial_state_e7 = 1/math.sqrt(2) * (basis[int(e7_1, 2)] + basis[int(e7_2, 2)])

initial_state_e8 = 1/math.sqrt(2) * (basis[int(e8_1, 2)] - basis[int(e8_2, 2)])

initial_state_e9 = 1/math.sqrt(2) * (basis[int(e9_1, 2)] - basis[int(e9_2, 2)])

initial_state_e10 = 1/math.sqrt(2) * (basis[int(e10_1, 2)] + basis[int(e10_2, 2)])

initial_state_e11 = 1/2 * (basis[int(e11_1, 2)] + basis[int(e11_2, 2)] 
                           - basis[int(e11_3, 2)] - basis[int(e11_4, 2)])

initial_state_e12 = 1/math.sqrt(2) * (basis[int(e12_1, 2)] - basis[int(e12_2, 2)])

initial_state_e13 = 1/math.sqrt(2) * (basis[int(e13_1, 2)] - basis[int(e13_2, 2)])

initial_state_e14 = 1/2 * (basis[int(e14_1, 2)] - basis[int(e14_2, 2)] 
                           - basis[int(e14_3, 2)] + basis[int(e14_4, 2)])

initial_state_e15 = basis[int(e15_1, 2)]

# The Pauli string representation of the Hamiltonian, particle number operator, and total spin z component operator
H_Pauli = [[-t * 0.5, 'X0,X1,I2,I3'], [-t * 0.5, 'Y0,Y1,I2,I3'], 
           [-t * 0.5, 'I0,I1,X2,X3'], [-t * 0.5, 'I0,I1,Y2,Y3'], 
           [U * 0.25, 'I0,I1,I2,I3'], [-U * 0.25, 'I0,I1,Z2,I3'], 
           [-U * 0.25, 'Z0,I1,I2,I3'], [U * 0.25, 'Z0,I1,Z2,I3'], 
           [U * 0.25, 'I0,I1,I2,I3'], [-U * 0.25, 'I0,I1,I2,Z3'], 
           [-U * 0.25, 'I0,Z1,I2,I3'], [U * 0.25, 'I0,Z1,I2,Z3']]

NumOp_Pauli = [[0.5, 'I0,I1,I2,I3'], [-0.5, 'Z0,I1,I2,I3'], 
               [0.5, 'I0,I1,I2,I3'], [-0.5, 'I0,I1,Z2,I3'], 
               [0.5, 'I0,I1,I2,I3'], [-0.5, 'I0,Z1,I2,I3'], 
               [0.5, 'I0,I1,I2,I3'], [-0.5, 'I0,I1,I2,Z3']]

Sz_Pauli = [[0.25, 'I0,I1,I2,I3'], [-0.25, 'Z0,I1,I2,I3'], 
            [-0.25, 'I0,I1,I2,I3'], [0.25, 'I0,I1,Z2,I3'], 
            [0.25, 'I0,I1,I2,I3'], [-0.25, 'I0,Z1,I2,I3'], 
            [-0.25, 'I0,I1,I2,I3'], [0.25, 'I0,I1,I2,Z3']]

# Matrix representation
dtype = paddle_quantum.get_dtype()
H_matrix = pauli_str_to_matrix(H_Pauli, N).astype(dtype)
NumOp_matrix = pauli_str_to_matrix(NumOp_Pauli, N).astype(dtype)
Sz_matrix = pauli_str_to_matrix(Sz_Pauli, N).astype(dtype)

# Convert to Tensor type
H_tensor = paddle.to_tensor(H_matrix)
NumOp_tensor = paddle.to_tensor(NumOp_matrix)
Sz_tensor = paddle.to_tensor(Sz_matrix)

# Classical methods to obtain the eigenstates of the Hamiltonian matrix
eigenvectors = numpy.array(
    [[ 0.0000000e+00+0.j, -0.0000000e+00+0.j, -0.0000000e+00+0.j,
   0.0000000e+00+0.j,  1.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j, -0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j],
 [ 0.0000000e+00+0.j, -7.0710677e-01+0.j, -0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
  -7.0710677e-01+0.j, -0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j],
 [ 0.0000000e+00+0.j, -7.0710677e-01+0.j, -0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   7.0710677e-01+0.j, -0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j],
 [ 0.0000000e+00+0.j, -0.0000000e+00+0.j, -0.0000000e+00+0.j,
   1.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j, -0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j],
 [ 0.0000000e+00+0.j, -0.0000000e+00+0.j, -7.0710677e-01+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j, -7.0710677e-01+0.j,
   0.0000000e+00+0.j, -0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j],
 [-3.7174803e-01+0.j, -0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
  -2.8381725e-17+0.j, -5.5511151e-16+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j, -0.0000000e+00+0.j,  7.0710677e-01+0.j,
  -5.5511151e-16+0.j,  0.0000000e+00+0.j, -6.0150093e-01+0.j,
   0.0000000e+00+0.j],
 [-6.0150093e-01+0.j, -0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   7.0710677e-01+0.j, -1.4623383e-16+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j, -0.0000000e+00+0.j, -5.5511151e-17+0.j,
   5.5774155e-16+0.j,  0.0000000e+00+0.j,  3.7174803e-01+0.j,
   0.0000000e+00+0.j],
 [ 0.0000000e+00+0.j, -0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
  -1.4485500e-16+0.j, -7.0710677e-01+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j, -0.0000000e+00+0.j, -1.2285131e-15+0.j,
   7.0710677e-01+0.j,  0.0000000e+00+0.j, -1.1102230e-15+0.j,
   0.0000000e+00+0.j],
 [ 0.0000000e+00+0.j,  0.0000000e+00+0.j, -7.0710677e-01+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  7.0710677e-01+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j],
 [-6.0150093e-01+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
  -7.0710677e-01+0.j, -2.2204460e-16+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  2.2204460e-16+0.j,
   6.6613381e-16+0.j,  0.0000000e+00+0.j,  3.7174803e-01+0.j,
   0.0000000e+00+0.j],
 [-3.7174803e-01+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
  -1.1193027e-17+0.j,  6.1062266e-16+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j, -7.0710677e-01+0.j,
  -1.0547119e-15+0.j,  0.0000000e+00+0.j, -6.0150093e-01+0.j,
   0.0000000e+00+0.j],
 [ 1.4462332e-17+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
  -4.3356907e-17+0.j, -7.0710677e-01+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j, -7.0776718e-16+0.j,
  -7.0710677e-01+0.j,  0.0000000e+00+0.j,  8.8860448e-16+0.j,
   0.0000000e+00+0.j],
 [ 0.0000000e+00+0.j, -0.0000000e+00+0.j, -0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  1.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j, -0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j],
 [ 0.0000000e+00+0.j, -0.0000000e+00+0.j, -0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j, -7.0710677e-01+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j, -7.0710677e-01+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j],
 [ 0.0000000e+00+0.j, -0.0000000e+00+0.j, -0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j, -7.0710677e-01+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  7.0710677e-01+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j],
 [ 0.0000000e+00+0.j, -0.0000000e+00+0.j, -0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j, -0.0000000e+00+0.j,  0.0000000e+00+0.j,
   0.0000000e+00+0.j,  0.0000000e+00+0.j,  0.0000000e+00+0.j,
   1.0000000e+00+0.j]])

start_time = time.time()

# Get desktop path
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")

# Specify folder path
folder_name = f"Data_1D_{file_corresponding[proc_sel]}"
folder_path = os.path.join(desktop_path, folder_name)

# Create folder (if the folder does not exist)
os.makedirs(folder_path, exist_ok=True)

for file_index in range(1, n_file + 1):
    file_value = os.path.join(folder_path, f"{file_corresponding[proc_sel]}_{file_index}.txt")
    file_params = os.path.join(folder_path, f"{file_corresponding[proc_sel]}_params_{file_index}.npy")

    with open(file_value, 'w') as file:

        # Create an instance
        VQE_symmetry = VQE_symmetry_class(depth=D, 
                                          width=W, 
                                          length_layer=L)

        # Adam optimizer
        optimizer = paddle.optimizer.Adam(learning_rate=Learning_Rate, 
                                          parameters=VQE_symmetry.parameters())

        # Optimization process
        for iter in range(1, iterations + 1):
            loss, H_psi, NumOp_psi, Sz_psi, cir = VQE_symmetry(proc_sel, 
                                                               H_tensor, 
                                                               NumOp_tensor, 
                                                               Sz_tensor)

            loss.backward()
            optimizer.minimize(loss)
            optimizer.clear_grad()

            # Save results to file
            selected_indices = states_corresponding[proc_sel]
            if iter % iter_step_size == 0 and proc_sel not in [Hea_g, Spat1_g, Spat2_g]:
                file.write(f'iterations: {iter}, loss: {loss.numpy()[0]:.{prec}f}\n')
            
            for i in range(16):
                if iter % iter_step_size == 0 and i in selected_indices:
                    file.write(f'iterations: {iter}, NumOp_psi{i}: {NumOp_psi[i].numpy()[0]:.{prec}f}\n')

            for i in range(16):
                if iter % iter_step_size == 0 and i in selected_indices:
                    file.write(f'iterations: {iter}, Sz_psi{i}: {Sz_psi[i].numpy()[0]:.{prec}f}\n')

            for i in range(16):
                if iter % iter_step_size == 0 and i in selected_indices:
                    file.write(f'iterations: {iter}, E{i}: {H_psi[i].numpy()[0]:.{prec}f}\n')

            # Compute fidelity
            U_theta = cir.unitary_matrix()

            ground_state = numpy.dot(U_theta, initial_state_g)

            initial_states = [globals()[f'initial_state_e{i}'] for i in range(1, 16)]
            excited_states = [numpy.dot(U_theta, initial_state_e) for initial_state_e in initial_states]

            ground_state_dagger = numpy.conj(ground_state)
            excited_states_dagger = [numpy.conj(state) for state in excited_states]

            fidelities = []

            inner_product_g = numpy.vdot(ground_state_dagger, eigenvectors[:, 0])
            fidelity_g = abs(inner_product_g)**2
            fidelities.append(fidelity_g)

            for i in range(15):
                inner_product_e = numpy.vdot(excited_states_dagger[i], eigenvectors[:, i + 1])
                fidelity_e = abs(inner_product_e)**2
                fidelities.append(fidelity_e)

            if iter % iter_step_size == 0 and proc_sel in [Hea_g, Spat1_g, Spat2_g]:
                file.write(f'iterations: {iter}, Fidelity of ground state: {fidelity_g:.{prec_fid}f}\n')
            elif iter % iter_step_size == 0 and proc_sel in [Hea_ae, Hea_ae_Sz, Spat2_en2]:
                file.write(f'iterations: {iter}, Fidelity of ground state: {fidelity_g:.{prec_fid}f}\n')
                for i in range(1, 16):
                    if i in selected_indices:
                        file.write(f'iterations: {iter}, Fidelity of excited state {i}: {fidelities[i]:.{prec_fid}f}\n')
            elif iter % iter_step_size == 0:
                for i in range(1, 16):
                    if i in selected_indices:
                        file.write(f'iterations: {iter}, Fidelity of excited state {i}: {fidelities[i]:.{prec_fid}f}\n')

        params_unrounded = VQE_symmetry.parameters()[0].numpy()
        params_prec = numpy.round(params_unrounded, decimals=8)    # parameters precision
        numpy.save(file_params, params_prec)

# runtime
end_time = time.time()
elapsed_time = end_time - start_time
print(f'Code execution time: {elapsed_time:.4f} seconds')


C:\software\Anaconda\envs\paddle_quantum_env\lib\site-packages\paddle\tensor\creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
C:\software\Anaconda\envs\paddle_quantum_env\lib\site-packages\paddle\tensor\creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


Select the process you want to compute. You can choose from (copy one of the lines):

Hardware efficient ansatz, ground state
Hardware efficient ansatz, all eigenstates
Hardware efficient ansatz, all eigenstates, Sz penalty term
Symmetry preserving ansatz type 1, ground state
Symmetry preserving ansatz type 2, ground state
Symmetry preserving ansatz type 2, Eigenstate, n=2
Symmetry preserving ansatz type 2, Eigenstate, n=1
Symmetry preserving ansatz type 2, Eigenstate, n=0
Symmetry preserving ansatz type 2, Eigenstate, n=3
Symmetry preserving ansatz type 2, Eigenstate, n=4

Note: If any other parameters need to be modified, please adjust them in the code. Symmetry preserving ansatz type 1, ground state


You selected: Symmetry preserving ansatz type 1, ground state

Program is running...     The generated data is saved in the Data_1D_Spat1_g folder on the desktop.



C:\software\Anaconda\envs\paddle_quantum_env\lib\site-packages\paddle\fluid\framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:


Code execution time: 6388.9490 seconds
